In [261]:
import pandas as pd
UFO = '../Resources/UFO.csv'
UFO = pd.read_csv(UFO)
UFO.head()

C:\Users\dljoh\AppData\Local\Temp\ipykernel_26364\1409412935.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  UFO = pd.read_csv(UFO)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,Unnamed: 11
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,NaN
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,NaN
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,NaN
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,NaN
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,NaN


In [263]:
# Convert 'duration (seconds)' column to numeric, forcing errors to NaN to handle any non-numeric values
UFO['duration (seconds)'] = pd.to_numeric(UFO['duration (seconds)'], errors='coerce')

# Filter rows where duration is between 1 and  7200 (2hours) and clean
UFO = UFO[UFO['duration (seconds)'] >=1]
UFO = UFO[UFO['duration (seconds)'] <= 7200]
UFO = UFO.drop(columns=['Unnamed: 11'])
UFO = UFO.dropna()
UFO


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
5,10/10/1961 19:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595,-82.188889
7,10/10/1965 23:45,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.1175,-73.408333
...,...,...,...,...,...,...,...,...,...,...,...
88867,9/9/2013 21:15,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.1658333,-86.784444
88868,9/9/2013 22:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.6136111,-116.202500
88870,9/9/2013 22:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.2972222,-122.284444
88871,9/9/2013 22:20,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.9011111,-77.265556


In [265]:
import pandas as pd
import sqlite3
database_path = 'UFO.db'  # You can specify the full path if needed
conn = sqlite3.connect(database_path)
table_name = 'ufo_sightings'
UFO.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()
print("Data has been successfully saved to SQLite database:", database_path)

Data has been successfully saved to SQLite database: UFO.db


In [267]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
csv_path = Path("../Resources/UFO.csv")  # Update with your actual CSV file path
database_path = Path("../Resources/UFO.sqlite")
if csv_path.exists():
    ufo_data = pd.read_csv(csv_path)
else:
    raise FileNotFoundError("CSV file not found.")
engine = create_engine(f"sqlite:///{database_path}")
ufo_data.to_sql("UFO", engine, if_exists="replace", index=False)

print("Data loaded successfully into the database.")
with engine.connect() as conn:
    query = text("SELECT * FROM UFO LIMIT 5")  # Limit to 5 rows for a quick test
    data = conn.execute(query)
    
    for record in data:
        print(record)

C:\Users\dljoh\AppData\Local\Temp\ipykernel_26364\2475973632.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  ufo_data = pd.read_csv(csv_path)


Data loaded successfully into the database.
('10/10/1949 20:30', 'san marcos', 'tx', 'us', 'cylinder', '2700', '45 minutes', 'This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit', '4/27/2004', '29.8830556', -97.9411111, None)
('10/10/1949 21:00', 'lackland afb', 'tx', None, 'light', '7200', '1-2 hrs', '1949 Lackland AFB&#44 TX.  Lights racing across the sky &amp; making 90 degree turns on a dime.', '12/16/2005', '29.38421', -98.581082, None)
('10/10/1955 17:00', 'chester (uk/england)', None, 'gb', 'circle', '20', '20 seconds', 'Green/Orange circular disc over Chester&#44 England', '1/21/2008', '53.2', -2.916667, None)
('10/10/1956 21:00', 'edna', 'tx', 'us', 'circle', '20', '1/2 hour', 'My older brother and twin sister were leaving the only Edna theater at about 9 PM&#44...we had our bikes and I took a different route home', '1/17/2004', '28.9783333', -96.6458333, None)
('10/10/1960 20:00', 'kaneohe', '

In [269]:
print(UFO.info())

<class 'pandas.core.frame.DataFrame'>
Index: 65229 entries, 0 to 88872
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              65229 non-null  object 
 1   city                  65229 non-null  object 
 2   state                 65229 non-null  object 
 3   country               65229 non-null  object 
 4   shape                 65229 non-null  object 
 5   duration (seconds)    65229 non-null  float64
 6   duration (hours/min)  65229 non-null  object 
 7   comments              65229 non-null  object 
 8   date posted           65229 non-null  object 
 9   latitude              65229 non-null  object 
 10  longitude             65229 non-null  float64
dtypes: float64(2), object(9)
memory usage: 6.0+ MB
None


In [271]:
UFO['duration (seconds)'] = pd.to_numeric(UFO['duration (seconds)'], errors='coerce')

UFO['latitude'] = pd.to_numeric(UFO['latitude'], errors='coerce')

In [273]:
export_path = '../Resources/UFO_data.csv'
UFO.to_csv(export_path, index=False)

export_path

'../Resources/UFO_data.csv'

In [275]:
UFO = '../Resources/UFO_data.csv'
UFO = pd.read_csv(UFO)
UFO

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
2,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611
3,10/10/1961 19:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889
4,10/10/1965 23:45,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333
...,...,...,...,...,...,...,...,...,...,...,...
65224,9/9/2013 21:15,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444
65225,9/9/2013 22:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500
65226,9/9/2013 22:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444
65227,9/9/2013 22:20,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556


In [277]:
print(UFO.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65229 entries, 0 to 65228
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              65229 non-null  object 
 1   city                  65229 non-null  object 
 2   state                 65229 non-null  object 
 3   country               65229 non-null  object 
 4   shape                 65229 non-null  object 
 5   duration (seconds)    65229 non-null  float64
 6   duration (hours/min)  65229 non-null  object 
 7   comments              65229 non-null  object 
 8   date posted           65229 non-null  object 
 9   latitude              65229 non-null  float64
 10  longitude             65229 non-null  float64
dtypes: float64(3), object(8)
memory usage: 5.5+ MB
None


In [279]:
UFO[['date', 'time']] = UFO['datetime'].str.split(' ', expand=True)
UFO = UFO.drop(columns=['datetime'])
UFO.head()

,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,date,time
0,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,10/10/1949,20:30
1,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,10/10/1956,21:00
2,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,10/10/1960,20:00
3,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,10/10/1961,19:00
4,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333,10/10/1965,23:45


In [283]:
#Most popular day?
most_popular_date = UFO['date'].mode()[0]
date_counts = UFO['date'].value_counts().head()
most_popular_date, date_counts

('7/4/2010',
 date
 7/4/2010      180
 7/4/2012      165
 11/16/1999    158
 7/4/2013      157
 7/4/2011      130
 Name: count, dtype: int64)

In [285]:
#Take out holidays
UFO = UFO[~UFO['date'].str.contains('12/31', na=False)]
UFO = UFO[~UFO['date'].str.contains('1/1', na=False)]
UFO = UFO[~UFO['date'].str.contains('7/4', na=False)]
UFO

,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,date,time
0,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,10/10/1949,20:30
1,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,10/10/1956,21:00
2,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,10/10/1960,20:00
3,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,10/10/1961,19:00
4,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333,10/10/1965,23:45
...,...,...,...,...,...,...,...,...,...,...,...,...
65224,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444,9/9/2013,21:15
65225,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500,9/9/2013,22:00
65226,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444,9/9/2013,22:00
65227,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556,9/9/2013,22:20


In [287]:
# Find the most popular date by counting occurrences of each date in the 'date' column
most_popular_date = UFO['date'].mode()[0]

# Count occurrences for verification
date_counts = UFO['date'].value_counts().head()

most_popular_date, date_counts

('9/19/2009',
 date
 9/19/2009     109
 10/31/2004     79
 7/6/2013       73
 3/13/1997      62
 7/3/2011       61
 Name: count, dtype: int64)

In [289]:
export_path_filtered = '../Resources/UFO_data_no_holidays.csv'
UFO.to_csv(export_path_filtered, index=False)

export_path_filtered

'../Resources/UFO_data_no_holidays.csv'

In [291]:
UFO = '../Resources/UFO_data_no_holidays.csv'
UFO = pd.read_csv(UFO)
UFO

,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,date,time
0,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,10/10/1949,20:30
1,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,10/10/1956,21:00
2,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,10/10/1960,20:00
3,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,10/10/1961,19:00
4,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333,10/10/1965,23:45
...,...,...,...,...,...,...,...,...,...,...,...,...
57168,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444,9/9/2013,21:15
57169,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500,9/9/2013,22:00
57170,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444,9/9/2013,22:00
57171,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556,9/9/2013,22:20


In [293]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine

csv_path = Path("../Resources/UFO_data_no_holidays.csv")  # Update with your actual CSV file path
database_path = Path("../Resources/UFO.sqlite")

if csv_path.exists():
    ufo_data = pd.read_csv(csv_path)
else:
    raise FileNotFoundError("CSV file not found.")

engine = create_engine(f"sqlite:///{database_path}")

ufo_data.to_sql("UFO", engine, if_exists="replace", index=False)

print("Data loaded successfully into the database.")

with engine.connect() as conn:
    query = text("SELECT * FROM UFO LIMIT 5")  # Limit to 5 rows for a quick test
    data = conn.execute(query)
    
    for record in data:
        print(record)


Data loaded successfully into the database.
('san marcos', 'tx', 'us', 'cylinder', 2700.0, '45 minutes', 'This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit', '4/27/2004', 29.8830556, -97.9411111, '10/10/1949', '20:30')
('edna', 'tx', 'us', 'circle', 20.0, '1/2 hour', 'My older brother and twin sister were leaving the only Edna theater at about 9 PM&#44...we had our bikes and I took a different route home', '1/17/2004', 28.9783333, -96.6458333, '10/10/1956', '21:00')
('kaneohe', 'hi', 'us', 'light', 900.0, '15 minutes', 'AS a Marine 1st Lt. flying an FJ4B fighter/attack aircraft on a solo night exercise&#44 I was at 50&#44000&#39 in a &quot;clean&quot; aircraft (no ordinan', '1/22/2004', 21.4180556, -157.8036111, '10/10/1960', '20:00')
('bristol', 'tn', 'us', 'sphere', 300.0, '5 minutes', 'My father is now 89 my brother 52 the girl with us now 51 myself 49 and the other fellow which worked with my fath

In [295]:
import pandas as pd
from sqlalchemy import create_engine, text

database_path = "../Resources/UFO.sqlite"
engine = create_engine(f"sqlite:///{database_path}")

query = text("""
    SELECT shape, state, COUNT(*) AS count
    FROM UFO
    WHERE date = '9/19/2009'
    AND shape NOT IN ('unknown', 'light')
    GROUP BY shape, state
    HAVING count > 1
    ORDER BY count DESC
""")

with engine.connect() as conn:
    result = conn.execute(query)
    data = result.fetchall()

df = pd.DataFrame(data, columns=["Shape", "State", "Count"])
df



,Shape,State,Count
0,cone,pa,5
1,circle,pa,3
2,circle,me,2
3,circle,ny,2
4,cone,ct,2
5,cone,nj,2
6,cone,ny,2
7,fireball,pa,2
8,oval,pa,2
9,triangle,me,2


In [231]:
# Count occurrences of each city
city_counts = UFO['city'].value_counts()

# Display the most popular city and its count
most_popular_city = city_counts.idxmax()
most_popular_count = city_counts.max()

print("Most popular city:", most_popular_city, "with", most_popular_count, "occurrences.")


Most popular city: seattle with 416 occurrences.


In [297]:
# Get the top 5 cities and their occurrence counts
top_5_cities = UFO['city'].value_counts().head(5)
top_5_cities

city
seattle        416
phoenix        384
portland       315
las vegas      312
los angeles    300
Name: count, dtype: int64

In [299]:
# Get the top 5 cities and their occurrence counts
top_5_states = UFO['state'].value_counts().head(5)
top_5_states

state
ca    7368
fl    3214
wa    3178
tx    2900
ny    2488
Name: count, dtype: int64

In [301]:
date_state_counts = UFO.groupby(['date', 'state']).size().reset_index(name='counts')
most_popular_state_by_date = date_state_counts.loc[date_state_counts.groupby('date')['counts'].idxmax()]
most_popular_state_by_date

,date,state,counts
0,1/2/1950,ny,1
1,1/2/1997,tx,1
3,1/2/1999,wa,2
6,1/2/2000,ca,13
9,1/2/2001,ar,1
...,...,...,...
44091,9/9/2009,mi,3
44102,9/9/2010,fl,4
44112,9/9/2011,ca,3
44132,9/9/2012,tn,4


In [303]:
top_10 = most_popular_state_by_date.nlargest(10, 'counts')
top_10

,date,state,counts
5474,10/31/2004,il,66
15255,3/13/1997,az,50
43272,9/30/2005,il,38
37168,8/21/2004,il,36
41464,9/19/2009,pa,26
2022,10/1/2005,il,25
20947,4/29/2005,fl,21
17854,3/7/1998,fl,20
34183,7/6/2013,oh,19
34247,7/7/2000,ca,18


In [305]:
# Group by 'date' and 'state', then count occurrences
date_city_counts = UFO.groupby(['date', 'city']).size().reset_index(name='counts')
most_popular_city_by_date = date_city_counts.loc[date_city_counts.groupby('date')['counts'].idxmax()]
most_popular_city_by_date

,date,city,counts
0,1/2/1950,new york city,1
1,1/2/1997,austin,1
2,1/2/1999,seattle,1
5,1/2/2000,anaheim hills,1
22,1/2/2001,merrillan,1
...,...,...,...
55065,9/9/2009,akron,1
55088,9/9/2010,gainesville,2
55101,9/9/2011,chula vista,2
55124,9/9/2012,murfreesboro,2


In [307]:
# Sort the data by 'counts' in descending order and show the top 10 entries
top_10 = most_popular_city_by_date.nlargest(10, 'counts')
top_10

,date,city,counts
6958,10/31/2004,tinley park,54
46141,8/21/2004,tinley park,27
19211,3/13/1997,phoenix,20
54032,9/30/2005,tinley park,12
15975,2/16/2000,rockford,9
2504,10/1/2005,oak forest,8
10551,11/7/2005,navarre,8
24949,4/21/2008,phoenix,8
26039,4/29/2006,portland,8
42274,7/6/2013,columbus,8


In [311]:
# Remove rows where 'missle' appears in the 'comments' column
data = pd.read_csv('../Resources/UFO_data_no_holidays.csv')
data_cleaned = data[~data['comments'].str.contains('missle', case=False, na=False)]
data_cleaned

,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,date,time
0,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,10/10/1949,20:30
1,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,10/10/1956,21:00
2,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,10/10/1960,20:00
3,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,10/10/1961,19:00
4,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333,10/10/1965,23:45
...,...,...,...,...,...,...,...,...,...,...,...,...
57168,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444,9/9/2013,21:15
57169,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500,9/9/2013,22:00
57170,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444,9/9/2013,22:00
57171,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556,9/9/2013,22:20


In [313]:
UFO_no_missle = '../Resources/UFO_no_missle.csv'
data_cleaned.to_csv(UFO_no_missle, index=False)
UFO_no_missle

PermissionError: [Errno 13] Permission denied: '../Resources/UFO_no_missle.csv'

In [315]:
date_city_counts_nm = data_cleaned.groupby(['date', 'city']).size().reset_index(name='counts')
most_popular_city_by_date_nm = date_city_counts_nm.loc[date_city_counts_nm.groupby('date')['counts'].idxmax()]
most_popular_city_by_date_nm

,date,city,counts
0,1/2/1950,new york city,1
1,1/2/1997,austin,1
2,1/2/1999,seattle,1
5,1/2/2000,anaheim hills,1
22,1/2/2001,merrillan,1
...,...,...,...
55042,9/9/2009,akron,1
55065,9/9/2010,gainesville,2
55078,9/9/2011,chula vista,2
55101,9/9/2012,murfreesboro,2


In [317]:
top_10 = most_popular_city_by_date_nm.nlargest(10, 'counts')
top_10

,date,city,counts
6955,10/31/2004,tinley park,54
46128,8/21/2004,tinley park,27
19207,3/13/1997,phoenix,20
54010,9/30/2005,tinley park,12
15971,2/16/2000,rockford,9
2503,10/1/2005,oak forest,8
10547,11/7/2005,navarre,8
24942,4/21/2008,phoenix,8
26032,4/29/2006,portland,8
42264,7/6/2013,columbus,8


In [319]:
date_state_counts_nm = data_cleaned.groupby(['date', 'state']).size().reset_index(name='counts')
most_popular_state_by_date_nm = date_state_counts_nm.loc[date_state_counts_nm.groupby('date')['counts'].idxmax()]
most_popular_state_by_date_nm

,date,state,counts
0,1/2/1950,ny,1
1,1/2/1997,tx,1
3,1/2/1999,wa,2
6,1/2/2000,ca,13
9,1/2/2001,ar,1
...,...,...,...
44074,9/9/2009,mi,3
44085,9/9/2010,fl,4
44095,9/9/2011,ca,3
44115,9/9/2012,tn,4


In [321]:
top_10 = most_popular_state_by_date_nm.nlargest(10, 'counts')
top_10

,date,state,counts
5472,10/31/2004,il,66
15252,3/13/1997,az,50
43256,9/30/2005,il,38
37157,8/21/2004,il,36
41448,9/19/2009,pa,26
2021,10/1/2005,il,25
20942,4/29/2005,fl,21
17851,3/7/1998,fl,20
34175,7/6/2013,oh,19
34239,7/7/2000,ca,18
